# Libraries

In [1]:
import os
import time
import numpy as np
import torch
from datetime import datetime
from torch.utils.data import TensorDataset, DataLoader

from src.utils import (
    set_seed, get_device, print_h, 
    eval_window, eval_person_severity_voting, eval_person_majority_voting, eval_person_max_severity, 
    init_metrics, update_metrics, save_metrics_to_json,
)
from src.models import RNNInceptionTime

/home/mitlab/miniconda3/envs/alxxtexxr_py312_torch22/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [2]:
seed = 69
set_seed(seed)
device = get_device()
print("Device:", device)

# Data config
k_fold_dir = 'data/preprocessed/Ga_k10_w500_s250_v20250501213826'

# Training config
batch_size = 8
n_feat = 16
n_class = 4
window_size = 500
lr = 3e-4
n_epoch = 1

run_name_tag = k_fold_dir.split('/')[-1].rsplit('_v', 1)[0] + f'_e{n_epoch}'
print("Run name tag:", run_name_tag)

2025-05-20 21:57:45.123122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-20 21:57:45.141876: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-20 21:57:45.147818: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-20 21:57:45.162105: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-20 21:57:46.100887: W tensorflow/compiler/tf2

Random seed: 69
Device: cuda
Run name tag: Ga_k10_w500_s250_e1


# Training

In [3]:
# Set run name
run_name = f'RNNInceptionTime_bidirectional_{run_name_tag+'_' if run_name_tag else ''}v{datetime.now().strftime("%Y%m%d%H%M%S")}'
print("Run name:", run_name)

# Create save directory
save_dir = 'checkpoints/' + run_name
os.makedirs(save_dir, exist_ok=True)
print("Save directory:", save_dir)

print()

# Initialize evaluation metrics
metrics = {
    'window': init_metrics(['acc', 'f1', 'precision', 'recall', 'cm', 'train_loss', 'val_loss', 'train_time']),
    'person_majority_voting': init_metrics(['acc', 'f1', 'precision', 'recall', 'cm', 'train_loss', 'val_loss']),
    # 'person_severity_voting': init_metrics(['acc', 'f1', 'precision', 'recall', 'cm', 'train_loss', 'val_loss']),
    # 'person_max_severity': init_metrics(['acc', 'f1', 'precision', 'recall', 'cm', 'train_loss', 'val_loss']),
}

for fold_i_dir_name in sorted(os.listdir(k_fold_dir)):
    # ================================================================================================================================
    # FOLD
    # ================================================================================================================================
    fold_i_dir = os.path.join(k_fold_dir, fold_i_dir_name)
    print_h(fold_i_dir_name, 128)

    # ================================================================================================
    # DATA
    # ================================================================================================
    X_train_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_train_window.npy'))).float()
    y_train_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_train_window.npy'))).long()

    X_val_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_val_window.npy'))).float()
    y_val_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_val_window.npy'))).long()

    X_test_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_test_window.npy'))).float()
    y_test_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_test_window.npy'))).long()

    X_val_person = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_val_person.npy'))).float()
    y_val_person = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_val_person.npy'))).long()

    X_test_person = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_test_person.npy'))).float()
    y_test_person = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_test_person.npy'))).long()

    train_window_dataset = TensorDataset(X_train_window, y_train_window)
    val_window_dataset = TensorDataset(X_val_window, y_val_window)
    test_window_dataset = TensorDataset(X_test_window, y_test_window)
    
    val_person_dataset = TensorDataset(X_val_person, y_val_person)
    test_person_dataset = TensorDataset(X_test_person, y_test_person)

    train_dataloader = DataLoader(train_window_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_window_dataset, batch_size=batch_size, shuffle=False)
    test_dataloader = DataLoader(test_window_dataset, batch_size=batch_size, shuffle=False)
    
    # ================================================================================================
    # TRAINING
    # ================================================================================================
    print_h("TRAINING", 96)

    # Initialize model
    model = RNNInceptionTime(c_in=n_feat, c_out=n_class, seq_len=window_size, bidirectional=True).to(device)

    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion =  torch.nn.CrossEntropyLoss()
    
    # Swith the model to training mode
    model.train()
    
    # Loop training epochs
    global_val_loss_window_list = []
    global_val_loss_person_list = []
    global_train_loss_list = []
    global_train_time_list = []
    train_loss_list = []
    
    for epoch in range(n_epoch):
        start_time = time.time()
        
        # Loop training batches
        for iter, (X_train, y_train) in enumerate(train_dataloader):
            # Flush the computed gradients
            optimizer.zero_grad()
            
            X_train = X_train.to(device)
            y_train = y_train.to(device)
            
            # Feed forward the model
            X_train = X_train.permute(0, 2, 1)
            y_pred = model(X_train)

            # Compute training loss
            train_loss = criterion(y_pred, y_train)
            train_loss_list.append(train_loss)
                        
            # Backward pass the model
            train_loss.backward()
            
            # Update the model weights based on computed gradients
            optimizer.step()

        # Compute training time
        train_time = time.time() - start_time
        global_train_time_list.append(train_time)
        
        # ================================================================
        # VALIDATION
        # ================================================================
        avg_val_loss_window, acc_window, f1_window, *_ = eval_window(model, val_dataloader, criterion, average='weighted')
        avg_val_loss_person, acc_person, f1_person, *_ = eval_person_majority_voting(model, test_person_dataset, criterion=criterion, average='weighted',
                                                                                     window_size=window_size)
        
        global_val_loss_window_list.append(avg_val_loss_window)
        global_val_loss_person_list.append(avg_val_loss_person)
        
        # Compute the average training loss for each epoch
        avg_train_loss = sum(train_loss_list) / len(train_dataloader)
        global_train_loss_list.append(avg_train_loss.item())
        train_loss_list = []
        
        # ================================================================
        # LOGGING
        # ================================================================
        print(f"epoch: {epoch+1}, "
            f"train/loss: {avg_train_loss:.3f}, "
            f"val/loss_window: {avg_val_loss_window:.3f}, "
            f"val/acc_window: {acc_window:.3f}, "
            f"val/f1_window: {f1_window:.3f}, "
            f"val/loss_person: {avg_val_loss_person:.3f}, "
            f"val/acc_person: {acc_person:.3f}, "
            f"val/f1_person: {f1_person:.3f}, "
            f"train/time: {train_time:.1f}s"
        )
        
        # Switch the model back to training mode
        model.train()
    print()

    # ================================================================================================
    # EVALUATION
    # ================================================================================================
    print_h("EVALUATION", 96)
    
    # ================================================================
    # EVALUATION ON WINDOW DATA
    # ================================================================
    print_h("EVALUATION ON WINDOW DATA", 64)
    (
        _, 
        acc_window, 
        f1_window, 
        precision_window, 
        recall_window, 
        cm_window
    ) = eval_window(
        model, 
        test_dataloader, 
        average='weighted',
    )
    print("acc:", acc_window)
    print("f1:", f1_window)
    print("precision:", precision_window)
    print("recall:", recall_window)
    print("cm:\n", np.array(cm_window))
    print()

    # ================================================================
    # EVALUATION ON PERSON DATA BY MAJORITY VOTING
    # ================================================================
    print_h("EVALUATION ON PERSON DATA BY MAJORITY VOTING", 64)
    (
        _,
        acc_person_majority_voting,
        f1_person_majority_voting,
        precision_person_majority_voting,
        recall_person_majority_voting,
        cm_person_majority_voting,
        *_,
    ) = eval_person_majority_voting(
        model, 
        test_person_dataset, 
        criterion=None, 
        average='weighted',
        window_size=window_size, 
        debug=False,
        seed=seed,
    )
    print("acc:", acc_person_majority_voting)
    print("f1:", f1_person_majority_voting)
    print("precision:", precision_person_majority_voting)
    print("recall:", recall_person_majority_voting)
    print("cm:\n", np.array(cm_person_majority_voting))
    print()
    
    # ================================================================
    # EVALUATION ON PERSON DATA BY SEVERITY VOTING
    # ================================================================
    # print_h("EVALUATION ON PERSON DATA BY SEVERITY VOTING", 64)
    # (
    #     _, 
    #     acc_person_severity_voting, 
    #     f1_person_severity_voting, 
    #     precision_person_severity_voting, 
    #     recall_person_severity_voting, 
    #     cm_person_severity_voting,
    # ) = eval_person_severity_voting(
    #     model, 
    #     test_person_dataset, 
    #     criterion=None, 
    #     average='weighted',
    #     window_size=window_size, 
    #     debug=False,
    # )
    # print("acc:", acc_person_severity_voting)
    # print("f1:", f1_person_severity_voting)
    # print("precision:", precision_person_severity_voting)
    # print("recall:", recall_person_severity_voting)
    # print("cm:\n", np.array(cm_person_severity_voting))
    # print()

    # ================================================================
    # EVALUATION ON PERSON DATA BY MAX. SEVERITY
    # ================================================================
    # print_h("EVALUATION ON PERSON DATA BY MAX. SEVERITY", 64)
    # (
    #     _, 
    #     acc_person_max_severity, 
    #     f1_person_max_severity, 
    #     precision_person_max_severity, 
    #     recall_person_max_severity, 
    #     cm_person_max_severity,
    # ) = eval_person_max_severity(
    #     model, 
    #     test_person_dataset, 
    #     criterion=None, 
    #     average='weighted',
    #     window_size=window_size, 
    #     debug=False,
    # )
    # print("acc:", acc_person_max_severity)
    # print("f1:", f1_person_max_severity)
    # print("precision:", precision_person_max_severity)
    # print("recall:", recall_person_max_severity)
    # print("cm:\n", np.array(cm_person_max_severity))
    # print()

    # Add evaluation metrics for current fold
    in_metrics = {
        'person_majority_voting': {
            'acc': acc_person_majority_voting,
            'f1': f1_person_majority_voting,
            'precision': precision_person_majority_voting,
            'recall': recall_person_majority_voting,
            'cm': cm_person_majority_voting,
        },
        # 'person_severity_voting': {
        #     'acc': acc_person_severity_voting,
        #     'f1': f1_person_severity_voting,
        #     'precision': precision_person_severity_voting,
        #     'recall': recall_person_severity_voting,
        #     'cm': cm_person_severity_voting,
        # },
        # 'person_max_severity': {
        #     'acc': acc_person_max_severity,
        #     'f1': f1_person_max_severity,
        #     'precision': precision_person_max_severity,
        #     'recall': recall_person_max_severity,
        #     'cm': cm_person_max_severity,
        # },
        'window': {
            'acc': acc_window,
            'f1': f1_window,
            'precision': precision_window,
            'recall': recall_window,
            'cm': cm_window,
        },
    }

    for metric_type in in_metrics.keys():
        update_metrics(metrics[metric_type], in_metrics[metric_type])

    metrics['window']['train_loss']['folds'].append(global_train_loss_list)
    metrics['window']['val_loss']['folds'].append(global_val_loss_window_list)
    metrics['window']['train_time']['folds'].append(global_train_time_list)

    # ================================================================================================
    # CHECKPOINT SAVING
    # ================================================================================================
    save_path = os.path.join(save_dir, f'{fold_i_dir_name}.pth')
    torch.save(model.state_dict(), save_path)
    print(f"Checkpoint for {fold_i_dir_name} is saved to:", save_path)
    print()

    # DEBUG: Test for only one fold
    # break

save_metrics_to_json(metrics, save_dir, filename='_train_evaluation_metrics.json')
print("Evaluation metrics is saved in:", save_dir)

Run name: RNNInceptionTime_bidirectional_Ga_k10_w500_s250_e1_v20250520215746

                                                            fold_01                                                             
                                            TRAINING                                            
Random seed: 69

epoch: 1, train/loss: 0.561, val/loss_window: 0.227, val/acc_window: 0.930, val/f1_window: 0.928, val/loss_person: 0.552, val/acc_person: 0.875, val/f1_person: 0.818, train/time: 20.2s

                                           EVALUATION                                           
                   EVALUATION ON WINDOW DATA                    
acc: 0.9276693455797933
f1: 0.925897159298106
precision: 0.9321109935571289
recall: 0.9276693455797933
cm:
 [[304   0   0   0]
 [ 32 259  16   3]
 [  0   0 205   0]
 [  0   5   7  40]]

          EVALUATION ON PERSON DATA BY MAJORITY VOTING          
Random seed: 69

acc: 0.75
f1: 0.7424242424242424
precision: 0.7604166666666667
